In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time, math
import cmath
from matplotlib.colors import hsv_to_rgb
import matplotlib
import sys
from numba import jit, cuda, prange, guvectorize, complex128
import cv2
# np.set_printoptions(precision=3,suppress=True)


In [2]:
# !python3 -m pip install numba
# !git clone https://github.com/numba/numba.git

# %cd numba
# !python setup.py install
# !pip3 install .
# %cd ../

# !pip install --upgrade https://github.com/numba/numba.git
# !pip install git+https://github.com/numba/numba.git#egg=numba

# https://github.com/numba/numba.git
# git@github.com:numba/numba.git

# !pip install numba --upgrade

In [3]:
@jit(nopython=True)
def f(x):
    # # a,b,c,d=[1,3**0.5,1,1/(3*3**0.5)]
    # a,b,c,d,e,f=[1,1,1,1,1,1]    
    # return a*x**5+b*x**4+c*x**3+d*x**2+e*x+f
    # a,b,c,d,e,f,g,h,i,j,k=[1,1,1,1,1,1,1,1,1,1,1]
    # return a*x**10+b*x**9+c*x**8+d*x**7+e*x**6+f*x**5+g*x**4+h*x**3+i*x**2+j*x**1+k*x**0
    
    coef=[1]*15
    eq=0
    for i in range(len(coef)):
        eq+=coef[i]*x**(len(coef)-i-1)
    return eq

@jit(nopython=True)
def df(x):
    # a,b,c,d,e,f=[1,1,1,1,1,1]
    # return 5*a*x**4+4*b*x**3+3*c*x**2+2*d*x**1+e
    # a,b,c,d,e,f,g,h,i,j,k=[1,1,1,1,1,1,1,1,1,1,1]
    # return 10*a*x**9+9*b*x**8+8*c*x**7+7*d*x**6+6*e*x**5+5*f*x**4+4*g*x**3+3*h*x**2+2*i*x**1+j*x**0
    
    coef=[1]*15
    eq=0
    for i in range(len(coef)-1):
        eq+=coef[i]*(len(coef)-i-1)*x**(len(coef)-i-2)
    return eq



In [4]:
@jit(nopython=True)
def NewtonR(x):
    for i in range(500):
        x=x-f(x)/df(x)
        if(abs(f(x)/df(x))<=1e-3):
            break
    return x

In [5]:
@jit(nopython=True)
def gradientDecent(x):
    lr=0.01
    tol=1e-6
    for i in range(100):
        der=df(x)
        x=x-lr*der
        if(abs(der*lr)<=tol):
            break
    return x

In [6]:
@cuda.jit
def get_grid_cuda(Z,coef,lenCoef):
    i, j = cuda.grid(2)
    if i >=0 and i < Z.shape[0] and j >=0 and j < Z.shape[1]:     
        for _ in range(500):
            
            fx=0
            for _i in range(lenCoef):
                fx+=coef[_i]*Z[i,j]**(lenCoef-_i-1)

            dfx=0
            for _i in range(lenCoef-1):
                dfx+=coef[_i]*(lenCoef-_i-1)*Z[i,j]**(lenCoef-_i-2)
            
            # fx=Z[i,j]**3+Z[i,j]**2+Z[i,j]+1
            # dfx=3*Z[i,j]**2+2*Z[i,j]+1
            div=fx/dfx
            Z[i,j]=Z[i,j]-div
            if(abs(div)<1e-6):
                break

In [7]:
# @cuda.jit
# # @guvectorize([(complex128[:,:],complex128[:,:])], '()->()', target='cuda')
# def get_grid_cuda(Z):
#     i, j = cuda.grid(2)
#     if i >=0 and i < Z.shape[0] and j >=0 and j < Z.shape[1]:     
#         for i in range(10):
#             r=abs(Z[i,j])
#             exp=math.atan2(Z[i,j].imag,Z[i,j].real)
#             Z3=r**3*math.cos(3*exp)+r**3*math.sin(3*exp)*1j
#             Z2=r**2*math.cos(2*exp)+r**2*math.sin(2*exp)*1j
#             fx=Z3+Z2+Z[i,j]+1
#             dfx=3*Z2+2*Z[i,j]+1
            
#             div=((fx.real*dfx.real+fx.imag*dfx.imag)+1j*(fx.imag*dfx.real-fx.real*dfx.imag))/abs(dfx)**2
            
#             # if(abs(div)<1e-3):
#             #     break
#             Z[i,j]=Z[i,j]-div


In [8]:
@jit(nopython=True, parallel=True)
def get_grid(Z):
    for i in prange(len(Z)):
        for j in prange(len(Z[0])):
            Z[i,j]=NewtonR(Z[i,j])
    return Z

In [9]:
@jit(nopython=True, parallel=True)
def get_grid_GD(Z):
    for i in prange(len(Z)):
        for j in prange(len(Z[0])):
            Z[i,j]=gradientDecent(Z[i,j])
    return Z

In [ ]:
N=2**14
lim=1.15
limj=1.15j

x=np.linspace(-lim,lim,N,dtype=complex)
xj=np.linspace(-limj,limj,N,dtype=complex)
xx,yy=np.meshgrid(x,xj)
X=xx+yy
# Z=f(X)
# plt.contourf(X.real,X.imag,Z,levels=30)
# plt.show()
# plt.close()


# Z=np.copy(X)
# Z_out=np.copy(X)

# threadsperblock = (4, 4)
# blockspergrid_x = math.ceil(Z.shape[0] / threadsperblock[0])
# blockspergrid_y = math.ceil(Z.shape[1] / threadsperblock[1])
# blockspergrid = (blockspergrid_x, blockspergrid_y)

# start = time.time()
# # get_grid_cuda[blockspergrid, threadsperblock](Z)
# get_grid_cuda[blockspergrid, threadsperblock](Z)

# print("Time taken:",time.time() - start)

# plt.contourf(X.real,X.imag,Z,levels=30)
# plt.show()
# plt.close()

# roundOff=1
# minima=np.unique(Z.round(roundOff))
# print(minima)
# print(len(minima))

# i=1
# j=1
# r=(Z[i,j].real**2+Z[i,j].imag**2)**0.5
# exp=math.atan2(Z[i,j].imag,Z[i,j].real)
# Z3=r**3*math.cos(3*exp)+r**3*math.sin(3*exp)*1j
# Z2=r**2*math.cos(2*exp)+r**2*math.sin(2*exp)*1j
# Z1=r**1*math.cos(1*exp)+r**1*math.sin(1*exp)*1j

# fx=Z3+Z2+Z[i,j]+1
# dfx=3*Z2+2*Z[i,j]+1
# print(r)
# print(exp)
# print(Z3)
# print(Z2)
# print(Z1)
# print(Z[i,j])
# print("fx",fx)
# print("dxf",dfx)

# print("fx/dxf",fx/dfx)
# div=((fx.real*dfx.real+fx.imag*dfx.imag)+1j*(fx.imag*dfx.real-fx.real*dfx.imag))/(dfx.real**2+dfx.imag**2)
# print("div",div)
# print(abs(dfx))
# print((dfx.real**2+dfx.imag**2)**0.5)
  
# sys.exit()

Z=np.copy(X)

start = time.time()
Z=get_grid(Z)
print("Time taken:",time.time() - start)

# plt.contourf(X.real,X.imag,Z,levels=30)
# plt.show()
# plt.close()


roundOff=1
minima=np.unique(Z.round(roundOff))
print(minima)
print(len(minima))

Img=np.zeros((Z.shape[0],Z.shape[1],3)).astype(np.uint8)
# cmap = matplotlib.cm.get_cmap('viridis')
cmap = matplotlib.cm.get_cmap('inferno')

for i in  range(len(minima)):
    # indx=np.argwhere(Val.round(3)==minima[i])
    indx=np.argwhere(np.flip(Z,0).round(roundOff)==minima[i])
    rgba = cmap(i/(len(minima)-1))
    Img[indx[:,0],indx[:,1],:]=np.array(rgba[:-1])*255

for i in  range(len(minima)):   
    print(int((minima[i].real+lim)*N),int((minima[i].imag+abs(limj))*N))
    x,y=int((minima[i].real+lim)/(2*lim)*N),int((minima[i].imag+abs(limj))/(2*abs(limj))*N)    
    Img = cv2.circle(Img, (x,y), radius=0, color=(255, 255, 255), thickness=-1)

Img = cv2.circle(Img, (N//2,N//2), radius=0, color=(255, 255, 255), thickness=-1)

# plt.imshow(Img)
# plt.gca().set_aspect('equal')
cv2.imwrite('Fractal.png',cv2.cvtColor(Img, cv2.COLOR_RGB2BGR))
# plt.show()
# plt.close()

